In [1]:
import pandas as pd

data = "dfa-test.csv"

In [8]:
df = pd.read_csv(data, names=["date","status"])
df.describe()

,date,status
count,1287,1287
unique,1287,3
top,2022-04-07 16:58:01,No live chat agents are available
freq,1,990


In [24]:
df["hour"] = df.apply(lambda x: x.date.split(" ")[1].split(":")[0], axis=1)
df["hour_min"] = df.apply(lambda x: ":".join(x.date.split(" ")[1].split(":")[0:2]), axis=1)
df["status"] = df["status"].astype("category")
df["status_id"] = df["status"].cat.codes

In [25]:
df

,date,status,hour,hour_min,status_id
0,2022-04-06 10:59:01,No live chat agents are available,10,10:59,1
1,2022-04-06 11:00:02,No live chat agents are available,11,11:00,1
2,2022-04-06 11:01:01,No live chat agents are available,11,11:01,1
3,2022-04-06 11:02:01,Live chat is unavailable,11,11:02,0
4,2022-04-06 11:03:02,Live chat is unavailable,11,11:03,0
...,...,...,...,...,...
1282,2022-04-11 09:44:01,Start a chat,09,09:44,2
1283,2022-04-11 09:45:01,No live chat agents are available,09,09:45,1
1284,2022-04-11 09:46:01,No live chat agents are available,09,09:46,1
1285,2022-04-11 09:47:01,No live chat agents are available,09,09:47,1


In [32]:
df[["hour","hour_min", "status_id"]].groupby("status_id").describe()

hour                 hour_min                   
          count unique top freq    count unique    top freq
status_id                                                  
0           216      6  16   75      216    141  16:53    3
1           990      8  13  173      990    448  13:53    3
2            81      8  09   16       81     76  15:54    2

In [68]:
df[["hour_min", "status_id"]].groupby("hour_min").describe()

status_id                                             
             count      mean       std  min  25%  50%  75%  max
hour_min                                                       
09:00          3.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
09:01          3.0  0.666667  1.154701  0.0  0.0  0.0  1.0  2.0
09:02          3.0  0.666667  1.154701  0.0  0.0  0.0  1.0  2.0
09:03          3.0  0.333333  0.577350  0.0  0.0  0.0  0.5  1.0
09:04          3.0  0.666667  0.577350  0.0  0.5  1.0  1.0  1.0
...            ...       ...       ...  ...  ...  ...  ...  ...
16:55          3.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
16:56          3.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
16:57          3.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
16:58          3.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
16:59          3.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0

[480 rows x 8 columns]